In [1]:
# Speech emotion recognition on MSP-IMPROV dataset
# feature: egemaps
# Algorithm: as described in Bagus TA et al., "RNN-based dimenisional SER",
#                            ASJ Autumn 2019.
# 2019-12-24: Modified from ser_improv_egamaps, change input to HFS
#             Filter only natural interaction and improvisation data
# 2019-12-25: Updaate to adapt train/test data-label, change it accordingly
#             to be run in csl directory (although data in improv_loso/)
# 2020-05-26: update for SC paper

import numpy as np
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, \
                         Dropout, Flatten, LSTM, BatchNormalization, \
                         RNN, SimpleRNN, Activation
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error
import random as rn
import tensorflow as tf
# from attention_helper import AttentionDecoder
#from read_csv import load_features
# CSL Paper: Dimensional speech emotion recognition from acoustic and text
# Changelog:
# 2019-09-01: initial version
# 2019-10-06: optimizer MTL parameters with linear search (in progress)
# 2019-12-21: update file for CSL journal
#             this python use following feature: mean, std, silence
import codecs
import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.manifold import TSNE

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

#from keras_self_attention import SeqSelfAttention
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from attention_helper import AttentionDecoder
#from read_csv import load_features

import random as rn
import tensorflow as tf

# import gensim
# from gensim.models import Word2Vec
# from gensim.models.keyedvectors import KeyedVectors

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# loading file and label, choose the feature type first!
feat = np.load('../improv_loso/data/feat_hfs_mspin_train.npy')
feat_train = feat.reshape(feat.shape[0], 1, feat.shape[1])

feat_test = np.load('../improv_loso/data/feat_hfs_mspin_test.npy')
feat_test = feat_test.reshape(feat_test.shape[0], 1, feat_test.shape[1])

# load train and test ata
list_path_train = '../improv_loso/data/data_mspin_train.csv'
list_file = pd.read_csv(list_path_train, index_col=None)

vad_list = [list_file['v'], list_file['a'], list_file['d']]
vad_train = np.array(vad_list).T

list_path_test = '../improv_loso/data/data_mspin_test.csv'
list_test = pd.read_csv(list_path_test, index_col=None)

vad_test = [list_test ['v'], list_test['a'], list_test['d']]
vad_test = np.array(vad_test).T

feat = np.concatenate([feat_train, feat_test], axis=0)
vad = np.concatenate([vad_train, vad_test])

# uncomment the following to remove silence feature (HFS1)
feat = feat[:,:, :-1]

# load test data
print('Feature shape: ', feat.shape)
print('Label shape: ', vad.shape)


Feature shape:  (7166, 1, 46)
Label shape:  (7166, 3)


In [4]:
# load data - label
list_path_train = '../improv_loso/data/data_mspin_train.csv'
list_train = pd.read_csv(list_path_train, index_col=None)

# vad_train = [list_train['v'], list_train['a'], list_train['d']]
# vad_train = np.array(vad_train).T

list_path_test = '../improv_loso/data/data_mspin_test.csv'
list_test = pd.read_csv(list_path_test, index_col=None)

## Text processing train
list_file = pd.concat([list_train, list_test])
text = list_file['transcript']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index
nb_words = len(word_index) + 1
print('Found %s unique tokens' % len(word_index))

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_text = []
x_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

# x_text = np.load('improv_loso/data/x_text.npy')
# x_train_text = x_text[:5744] 
# x_test_text = x_text[5744:]

# vad = np.load('improv_loso/data/vad.npy')
# vad_train = vad[:5744]
# vad_test = vad[5744:]

EMBEDDING_DIM = 300
nb_words = 3438
MAX_SEQUENCE_LENGTH = 300

#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
file_loc =  '../../../dimensionalSM/data/glove.42B.300d.txt'
print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

Found 3427 unique tokens
../../../dimensionalSM/data/glove.42B.300d.txt
G Word embeddings: 1917494
G Null word embeddings: 167


In [5]:
## standardization
scaled_vad = True

if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

In [6]:
# split train/test
split = 5733

In [7]:
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(128, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(0.1)(net_speech)
    
    #text network
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net_text = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(input_text)
    net = Bidirectional(LSTM(128, return_sequences=True))(net_text)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    model_text = Dense(64)(net)

    # combined model
    model_combined = concatenate([model_speech, model_text])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    #model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_combined) for name in target_names]

    model = Model([input_speech, input_text], model_combined) 
    model.compile(loss=ccc_loss,
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model
# model2 = model()

In [8]:
# main function for for LOSO
def main(time):
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:split],x_text[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[split:],x_text[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model_1.predict([feat[4586:],x_text[4586:]], batch_size=32)
    np.save('5733npy/st_5733-npy'+str(time)+'.npy',  
             np.array(predict).reshape(3, 2580).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('5733npy/st_5733.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 99s 21ms/step - loss: 1.8314 - v_loss: 0.6650 - a_loss: 0.5629 - d_loss: 0.6014 - v_ccc: 0.3360 - a_ccc: 0.4359 - d_ccc: 0.3967 - val_loss: 1.8986 - val_v_loss: 0.7012 - val_a_loss: 0.6064 - val_d_loss: 0.5916 - val_v_ccc: 0.2987 - val_a_ccc: 0.3940 - val_d_ccc: 0.4086
Epoch 2/50
4586/4586 [==============================] - 90s 20ms/step - loss: 1.3387 - v_loss: 0.4918 - a_loss: 0.3760 - d_loss: 0.4720 - v_ccc: 0.5077 - a_ccc: 0.6254 - d_ccc: 0.5282 - val_loss: 1.5374 - val_v_loss: 0.6145 - val_a_loss: 0.4235 - val_d_loss: 0.4994 - val_v_ccc: 0.3849 - val_a_ccc: 0.5773 - val_d_ccc: 0.5005
Epoch 3/50
4586/4586 [==============================] - 90s 20ms/step - loss: 1.1897 - v_loss: 0.4072 - a_loss: 0.3461 - d_loss: 0.4417 - v_ccc: 0.5924 - a_ccc: 0.6558 - d_ccc: 0.5621 - val_loss: 1.5223 - val_v_loss: 0

Epoch 11/50
4586/4586 [==============================] - 90s 20ms/step - loss: 0.5741 - v_loss: 0.1719 - a_loss: 0.1873 - d_loss: 0.2176 - v_ccc: 0.8291 - a_ccc: 0.8147 - d_ccc: 0.7821 - val_loss: 1.8022 - val_v_loss: 0.6494 - val_a_loss: 0.4871 - val_d_loss: 0.6651 - val_v_ccc: 0.3498 - val_a_ccc: 0.5128 - val_d_ccc: 0.3352
Epoch 12/50
4586/4586 [==============================] - 90s 20ms/step - loss: 0.5252 - v_loss: 0.1584 - a_loss: 0.1749 - d_loss: 0.1930 - v_ccc: 0.8421 - a_ccc: 0.8253 - d_ccc: 0.8074 - val_loss: 1.8141 - val_v_loss: 0.6630 - val_a_loss: 0.5177 - val_d_loss: 0.6328 - val_v_ccc: 0.3364 - val_a_ccc: 0.4820 - val_d_ccc: 0.3675
Epoch 13/50
1433/1433 [==============================] - 8s 6ms/step
[0.30025583505630493, 0.4644298851490021, 0.3732803761959076]
1
Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 98s 21ms/step - loss: 1.7994 - v_loss: 0.6690 - a_loss: 0.5318 - d_loss: 0.5958 - v_ccc: 0.3290 - a_ccc: 0.46

4586/4586 [==============================] - 90s 20ms/step - loss: 0.6781 - v_loss: 0.2015 - a_loss: 0.2206 - d_loss: 0.2563 - v_ccc: 0.7979 - a_ccc: 0.7807 - d_ccc: 0.7433 - val_loss: 1.8087 - val_v_loss: 0.6995 - val_a_loss: 0.4722 - val_d_loss: 0.6361 - val_v_ccc: 0.2990 - val_a_ccc: 0.5281 - val_d_ccc: 0.3641
Epoch 10/50
4586/4586 [==============================] - 90s 20ms/step - loss: 0.6147 - v_loss: 0.1852 - a_loss: 0.1982 - d_loss: 0.2317 - v_ccc: 0.8151 - a_ccc: 0.8023 - d_ccc: 0.7679 - val_loss: 1.6892 - val_v_loss: 0.6566 - val_a_loss: 0.4669 - val_d_loss: 0.5657 - val_v_ccc: 0.3422 - val_a_ccc: 0.5333 - val_d_ccc: 0.4353
Epoch 11/50
4586/4586 [==============================] - 90s 20ms/step - loss: 0.5524 - v_loss: 0.1655 - a_loss: 0.1812 - d_loss: 0.2052 - v_ccc: 0.8341 - a_ccc: 0.8185 - d_ccc: 0.7950 - val_loss: 1.7355 - val_v_loss: 0.6731 - val_a_loss: 0.4737 - val_d_loss: 0.5889 - val_v_ccc: 0.3257 - val_a_ccc: 0.5267 - val_d_ccc: 0.4120
Epoch 12/50
1433/1433 [========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4586/4586 [==============================] - 91s 20ms/step - loss: 1.2022 - v_loss: 0.4136 - a_loss: 0.3509 - d_loss: 0.4387 - v_ccc: 0.5860 - a_ccc: 0.6502 - d_ccc: 0.5616 - val_loss: 1.7813 - val_v_loss: 0.7641 - val_a_loss: 0.4417 - val_d_loss: 0.5746 - val_v_ccc: 0.2345 - val_a_ccc: 0.5585 - val_d_ccc: 0.4257
Epoch 4/50
4586/4586 [==============================] - 92s 20ms/step - loss: 1.0854 - v_loss: 0.3555 - a_loss: 0.3244 - d_loss: 0.4083 - v_ccc: 0.6455 - a_ccc: 0.6773 - d_ccc: 0.5918 - val_loss: 1.5430 - val_v_loss: 0.6167 - val_a_loss: 0.4184 - val_d_loss: 0.5072 - val_v_ccc: 0.3819 - val_a_ccc: 0.5818 - val_d_ccc: 0.4933
Epoch 5/50
4586/4586 [==============================] - 92s 20ms/step - loss: 0.9907 - v_loss: 0.3083 - a_loss: 0.3003 - d_loss: 0.3791 - v_ccc: 0.6913 - a_ccc: 0.6985 - d_ccc: 0.6196 - val_loss: 1.6108 - val_v_loss: 0.6430 - val_a_loss: 0.4432 - val_d_loss: 0.5240 - val_v_ccc: 0.3555 - val_a_ccc: 0.5572 - val_d_ccc: 0.4764
Epoch 6/50
4586/4586 [===========

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4586/4586 [==============================] - 89s 19ms/step - loss: 0.9926 - v_loss: 0.3159 - a_loss: 0.2958 - d_loss: 0.3810 - v_ccc: 0.6839 - a_ccc: 0.7042 - d_ccc: 0.6192 - val_loss: 1.5308 - val_v_loss: 0.5951 - val_a_loss: 0.4304 - val_d_loss: 0.5048 - val_v_ccc: 0.4036 - val_a_ccc: 0.5699 - val_d_ccc: 0.4957
Epoch 6/50
4586/4586 [==============================] - 89s 19ms/step - loss: 0.8939 - v_loss: 0.2742 - a_loss: 0.2780 - d_loss: 0.3417 - v_ccc: 0.7255 - a_ccc: 0.7225 - d_ccc: 0.6580 - val_loss: 1.5341 - val_v_loss: 0.6027 - val_a_loss: 0.4297 - val_d_loss: 0.5006 - val_v_ccc: 0.3958 - val_a_ccc: 0.5706 - val_d_ccc: 0.4995
Epoch 7/50
4586/4586 [==============================] - 100s 22ms/step - loss: 0.8245 - v_loss: 0.2501 - a_loss: 0.2611 - d_loss: 0.3172 - v_ccc: 0.7507 - a_ccc: 0.7403 - d_ccc: 0.6845 - val_loss: 1.6066 - val_v_loss: 0.6151 - val_a_loss: 0.4628 - val_d_loss: 0.5283 - val_v_ccc: 0.3838 - val_a_ccc: 0.5375 - val_d_ccc: 0.4721
Epoch 8/50
4586/4586 [==========

1433/1433 [==============================] - 9s 6ms/step
[0.26908591389656067, 0.5150635242462158, 0.37644162774086]
15
Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 100s 22ms/step - loss: 1.8520 - v_loss: 0.6844 - a_loss: 0.5641 - d_loss: 0.6067 - v_ccc: 0.3163 - a_ccc: 0.4382 - d_ccc: 0.3935 - val_loss: 1.6881 - val_v_loss: 0.6488 - val_a_loss: 0.4755 - val_d_loss: 0.5636 - val_v_ccc: 0.3506 - val_a_ccc: 0.5248 - val_d_ccc: 0.4365
Epoch 2/50
4586/4586 [==============================] - 85s 18ms/step - loss: 1.3449 - v_loss: 0.5004 - a_loss: 0.3773 - d_loss: 0.4644 - v_ccc: 0.4981 - a_ccc: 0.6220 - d_ccc: 0.5350 - val_loss: 1.6053 - val_v_loss: 0.6706 - val_a_loss: 0.4281 - val_d_loss: 0.5063 - val_v_ccc: 0.3287 - val_a_ccc: 0.5723 - val_d_ccc: 0.4936
Epoch 3/50
4586/4586 [==============================] - 95s 21ms/step - loss: 1.2047 - v_loss: 0.4215 - a_loss: 0.3479 - d_loss: 0.4369 - v_ccc: 0.5794 - a_ccc: 0.6515 - d_ccc: 0.

4586/4586 [==============================] - 92s 20ms/step - loss: 0.6469 - v_loss: 0.1970 - a_loss: 0.2126 - d_loss: 0.2427 - v_ccc: 0.8053 - a_ccc: 0.7888 - d_ccc: 0.7590 - val_loss: 1.8144 - val_v_loss: 0.6731 - val_a_loss: 0.4734 - val_d_loss: 0.6674 - val_v_ccc: 0.3258 - val_a_ccc: 0.5267 - val_d_ccc: 0.3330
Epoch 11/50
4586/4586 [==============================] - 92s 20ms/step - loss: 0.5934 - v_loss: 0.1803 - a_loss: 0.1957 - d_loss: 0.2186 - v_ccc: 0.8200 - a_ccc: 0.8054 - d_ccc: 0.7812 - val_loss: 1.7328 - val_v_loss: 0.6433 - val_a_loss: 0.4828 - val_d_loss: 0.6054 - val_v_ccc: 0.3554 - val_a_ccc: 0.5173 - val_d_ccc: 0.3945
Epoch 12/50
4586/4586 [==============================] - 92s 20ms/step - loss: 0.5507 - v_loss: 0.1639 - a_loss: 0.1854 - d_loss: 0.2092 - v_ccc: 0.8369 - a_ccc: 0.8194 - d_ccc: 0.7931 - val_loss: 1.7272 - val_v_loss: 0.6394 - val_a_loss: 0.4737 - val_d_loss: 0.6141 - val_v_ccc: 0.3598 - val_a_ccc: 0.5264 - val_d_ccc: 0.3866
Epoch 13/50
4586/4586 [========

4586/4586 [==============================] - 91s 20ms/step - loss: 0.7825 - v_loss: 0.2361 - a_loss: 0.2488 - d_loss: 0.2985 - v_ccc: 0.7643 - a_ccc: 0.7516 - d_ccc: 0.7017 - val_loss: 1.6520 - val_v_loss: 0.6546 - val_a_loss: 0.4446 - val_d_loss: 0.5518 - val_v_ccc: 0.3442 - val_a_ccc: 0.5556 - val_d_ccc: 0.4482
Epoch 9/50
4586/4586 [==============================] - 92s 20ms/step - loss: 0.7114 - v_loss: 0.2124 - a_loss: 0.2296 - d_loss: 0.2734 - v_ccc: 0.7876 - a_ccc: 0.7720 - d_ccc: 0.7290 - val_loss: 1.8990 - val_v_loss: 0.7322 - val_a_loss: 0.5157 - val_d_loss: 0.6514 - val_v_ccc: 0.2672 - val_a_ccc: 0.4846 - val_d_ccc: 0.3491
Epoch 10/50
4586/4586 [==============================] - 92s 20ms/step - loss: 0.6434 - v_loss: 0.1935 - a_loss: 0.2060 - d_loss: 0.2432 - v_ccc: 0.8058 - a_ccc: 0.7941 - d_ccc: 0.7567 - val_loss: 1.8105 - val_v_loss: 0.6539 - val_a_loss: 0.4957 - val_d_loss: 0.6610 - val_v_ccc: 0.3456 - val_a_ccc: 0.5043 - val_d_ccc: 0.3397
Epoch 11/50
4586/4586 [=========

In [9]:
# earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)
# hist = model2.fit([feat[:split],x_text[:split]], vad[:split].T.tolist(), batch_size=32,
#                   validation_split=0.2, epochs=50, verbose=1, shuffle=True,
#                   callbacks=[earlystop])

# # outs =loss, mse_v, mse_a, mse_d, ccc_v, ccc_a, ccc_d
# # test data start from 5744/split
# metrik2 = model2.evaluate([feat[split:],x_text[split:]], vad[split:].T.tolist())#, a[8000:], d[8000:]])
# print(metrik2[-3:])

# # # make prediction
# # feat_predict = feat[4596:]
# # vad_predict = vad[4596:]

# predict2 = model2.predict([feat[4596:],x_text[4596:]], batch_size=32)
# np.save('st_glove_npy_loso',  np.array(predict2).reshape(3, 2570).T)